# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,2.91,97,96,2.56,CL,1691632834
1,1,invercargill,-46.4000,168.3500,6.68,76,22,0.45,NZ,1691633102
2,2,port saint john's,-31.6229,29.5448,16.37,87,49,3.88,ZA,1691633103
3,3,spittal an der drau,46.8000,13.5000,16.36,95,62,1.57,AT,1691633103
4,4,waitangi,-43.9535,-176.5597,11.57,78,5,1.34,NZ,1691633103


In [15]:
%%capture --no-display

# Configure the map plot
all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True, 
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = "Country",
)


# Display the map
all_city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp0 = city_data_df.loc[city_data_df["Max Temp"]<27,:]
city_temp =city_temp0.loc[city_temp0["Max Temp"]>21,:]
city_wind = city_temp.loc[city_temp["Wind Speed"]<4.5,:]
city_cloud = city_wind.loc[city_wind["Cloudiness"]==0,:]
city_cloud

# Drop any rows with null values
city_ideal = city_cloud.dropna()

city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,belmonte,-15.8631,-38.8828,21.79,72,0,3.31,BR,1691633108
31,31,bethel,41.3712,-73.4140,24.20,73,0,3.60,US,1691633047
93,93,fort dodge,42.4975,-94.1680,22.52,88,0,0.00,US,1691633120
153,153,urzhar,47.0930,81.6294,25.34,41,0,2.59,KZ,1691633151
169,169,westport,41.1415,-73.3579,25.99,67,0,2.57,US,1691633153
344,344,turkmenabat,39.0733,63.5786,23.99,50,0,2.06,TM,1691633178
352,352,ballina,-28.8667,153.5667,22.66,43,0,2.41,AU,1691633180
365,365,alice springs,-23.7000,133.8833,24.75,19,0,3.60,AU,1691633182
423,423,brookings,44.3114,-96.7984,23.19,78,0,0.00,US,1691633191
480,480,erzincan,39.7500,39.5000,23.61,38,0,1.86,TR,1691632921


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
30,belmonte,BR,-15.8631,-38.8828,72,
31,bethel,US,41.3712,-73.4140,73,
93,fort dodge,US,42.4975,-94.1680,88,
153,urzhar,KZ,47.0930,81.6294,41,
169,westport,US,41.1415,-73.3579,67,
344,turkmenabat,TM,39.0733,63.5786,50,
352,ballina,AU,-28.8667,153.5667,43,
365,alice springs,AU,-23.7000,133.8833,19,
423,brookings,US,44.3114,-96.7984,78,
480,erzincan,TR,39.7500,39.5000,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
        "apiKey":geoapify_key,
        "categories":"accomodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
belmonte - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
fort dodge - nearest hotel: No hotel found
urzhar - nearest hotel: No hotel found
westport - nearest hotel: No hotel found
turkmenabat - nearest hotel: No hotel found
ballina - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
brookings - nearest hotel: No hotel found
erzincan - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
bojnurd - nearest hotel: No hotel found
arvayheer - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
el burgo de osma - nearest hotel: No hotel found
izberbash - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
30,belmonte,BR,-15.8631,-38.8828,72,No hotel found
31,bethel,US,41.3712,-73.4140,73,No hotel found
93,fort dodge,US,42.4975,-94.1680,88,No hotel found
153,urzhar,KZ,47.0930,81.6294,41,No hotel found
169,westport,US,41.1415,-73.3579,67,No hotel found
344,turkmenabat,TM,39.0733,63.5786,50,No hotel found
352,ballina,AU,-28.8667,153.5667,43,No hotel found
365,alice springs,AU,-23.7000,133.8833,19,No hotel found
423,brookings,US,44.3114,-96.7984,78,No hotel found
480,erzincan,TR,39.7500,39.5000,38,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
hotel_map= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)